In [1]:

#a player's showup score is the integer total of games within
#a span of weeks in which they performed at or beyond a threshold

import pandas as pd
import os

#this version starts with weekly fantasy player data provided by (https://github.com/fantasydatapros/data) - endless gratitude
datafiles = os.listdir('fantasydata')
wknum = len(datafiles)

#every performance a tuple - (player, position, points, week)
data = []
cnt=0
for filename in datafiles:
    wks = filename[:-5]
    cnt = cnt+1
    df = pd.read_excel(datadir + '\\' + filename)
    plyr = df['Player']
    pos = df['Pos']     
    pnts = df['HalfPPRFantasyPoints']
    wks = len(plyr)*[wks]
    cnts = len(plyr)*[cnt]
    plypts = zip(plyr, pnts, wks, pos, cnts)
    data.append(plypts)

player=[]
points=[]
weeks=[]
QB=[]
QBp=[]
qbbs=0
RB=[]
RBp=[]
rbbs=0
WR=[]
WRp=[]
wrrs=0
TE=[]
TEp=[]
tees=0
#collecting the performances for each player - each player gets list of weekwise performances
for pltup in data:
    plt = list(pltup)
    
    for pl in plt:
        plyr = pl[0]
        pnts = pl[1]
        week = pl[2]
        pos = pl[3]
        cnt = pl[4]
    
        if week not in weeks:
            weeks.append(week)
    
        if plyr not in player:
            player.append(plyr)
            pntlis = wknum*[0]
            pntlis[cnt-1] = pnts
            points.append(pntlis)
        
        if pos == 'QB' and plyr not in QB:
            qbbs=qbbs+1
            QB.append(plyr)
            QBp.append(pntlis)
            QBp[qbbs-1][cnt-1] = pnts

        elif pos == 'RB' and plyr not in RB:
            rbbs=rbbs+1
            RB.append(plyr)
            RBp.append(pntlis)
            RBp[rbbs-1][cnt-1] = pnts

        elif pos == 'WR' and plyr not in WR:
            wrrs=wrrs+1
            WR.append(plyr)
            WRp.append(pntlis)
            WRp[wrrs-1][cnt-1] = pnts

        elif pos == 'TE' and plyr not in TE:
            tees=tees+1
            TE.append(plyr)
            TEp.append(pntlis)
            TEp[tees-1][cnt-1] = pnts

        index = player.index(plyr)
        points[index][cnt-1] = pnts

#index - player, data - points, columns - week
df = pd.DataFrame(points, player, columns=weeks)
dfqb = pd.DataFrame(QBp, QB, columns=weeks)
dfrb = pd.DataFrame(RBp, RB, columns=weeks)
dfwr = pd.DataFrame(WRp, WR, columns=weeks)
dfte = pd.DataFrame(TEp, TE, columns=weeks)



In [ ]:
##################################################################

In [4]:


#calculate show up scores during the selected season(s)
#each player gets one showup integer tally (+1) for each individual game performance that
#resulted in equal or more HalfPPR fantasy points than the threshold
thresh=30 #point threshold for games
styr=2017 #how far back does the dataframe go?
yr1=2020 #start season span
yr2=2021 #stop season span

if yr1 == yr2:
    analypath='showuppath\\'+str(yr1)
    
else:
    analypath='showuppath\\'+str(yr1)+'-'+str(yr2)

os.mkdir(analypath)

allname ='all'
qbname ='QBs'
rbname ='RBs'
wrname ='WRs'
tename ='TEs'

#adjusting for the recent change in season length
if yr1 < 2021 and yr2 < 2021:
    sy=(yr1-styr)*17
    ey=sy+(yr2-yr1+1)*17

if yr1 < 2021 and yr2 >= 2021:
    sy=(yr1-styr)*17
    ey=sy+(2021-yr1+1)*17+(yr2-2021+1)*18

if yr1 == 2021:
    sy=(yr1-styr)*17
    ey=sy+(yr2-yr1+1)*18

if yr1 > 2021 and yr2 > 2021:
    sy=(yr1-styr)*18
    ey=sy+(yr2-yr1+1)*18
sy=int(sy)
ey=int(ey)


for thr in range(10,thresh+1):#point threshold for scores between 

    dfs = df.iloc[:,sy:ey][df >= thr]
    dfs[dfs > 0] = 1
    dfs['Total'] = dfs.sum(axis=1)
    dfs = dfs.sort_values(by=['Total'], ascending=False)

    dfqbs = dfqb.iloc[:,sy:ey][dfqb >= thr]
    dfqbs[dfqbs > 0] = 1
    dfqbs['Total'] = dfqbs.sum(axis=1)
    dfqbs = dfqbs.sort_values(by=['Total'], ascending=False)

    dfrbs = dfrb.iloc[:,sy:ey][dfrb >= thr]
    dfrbs[dfrbs > 0] = 1
    dfrbs['Total'] = dfrbs.sum(axis=1)
    dfrbs = dfrbs.sort_values(by=['Total'], ascending=False)

    dfwrs = dfwr.iloc[:,sy:ey][dfwr >= thr]
    dfwrs[dfwrs > 0] = 1
    dfwrs['Total'] = dfwrs.sum(axis=1)
    dfwrs = dfwrs.sort_values(by=['Total'], ascending=False)

    dftes = dfte.iloc[:,sy:ey][dfte >= thr]
    dftes[dftes > 0] = 1
    dftes['Total'] = dftes.sum(axis=1)
    dftes = dftes.sort_values(by=['Total'], ascending=False)

    with pd.ExcelWriter(analypath+'\showup_'+str(thr)+'.xlsx', engine='openpyxl') as writer:
        dfs.to_excel(writer, sheet_name=allname)
        dfqbs.to_excel(writer, sheet_name=qbname)
        dfrbs.to_excel(writer, sheet_name=rbname)
        dfwrs.to_excel(writer, sheet_name=wrname)
        dftes.to_excel(writer, sheet_name=tename)

        book = writer.book
    writer.save

